## Project 2 -Model Data using Cassandra
> Name: Sharun Garg  
> Student ID: 200493338

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [2]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create & Set Keyspace

In [3]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS soundcloud 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)

In [4]:
try:
    session.set_keyspace('soundcloud')
except Exception as e:
    print(e)

## Query 1

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4

#### Table 1
**Columns:** 
1. session_number
2. item_in_session_number
3. artist_name
4. song_title
5. length

**Primary Key:**
- (session_number, item_in_session_number)

In [5]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command  
table1 = "artist_song_length_table"
query = "CREATE TABLE IF NOT EXISTS {}".format(table1)
query = query + "(session_number int, item_in_session_number int, artist_name text, song_title text, length float, PRIMARY KEY (session_number, item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Let's insert our data into of table

In [6]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO artist_song_length_table (session_number, item_in_session_number, artist_name, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))

### Validate our Data Model using a SELECT

In [7]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
val_query = "SELECT artist_name, song_title, length FROM artist_song_length_table WHERE session_number = 338 AND item_in_session_number = 4"
try:
    results = session.execute(val_query)
except Exception as e:
    print(e)
    
for result in results:
    print ("Artist: {}\nSong: {}\nLength: {}\n".format(result.artist_name,
                                                      result.song_title,
                                                      result.length))
    

Artist: Faithless
Song: Music Matters (Mark Knight Dub)
Length: 495.30731201171875



## Qeury 2

### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182

#### Table 2
**Columns:** 
1. user_id
2. session_number
3. item_in_session_number
4. artist_name
5. song_title
6. fname
7. lname

**Partition Key:**
- (session_number, user_id)

**Clustering Key**
- item_in_session_number

In [8]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command  

table2 = "artist_song_user_table"
query = "CREATE TABLE IF NOT EXISTS {}".format(table2)
query = query + "(user_id int, session_number int, item_in_session_number int, artist_name text, song_title text, fname text, lname text, PRIMARY KEY ((session_number, user_id), item_in_session_number))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data2.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO artist_song_user_table (user_id, session_number, item_in_session_number, artist_name, song_title, fname, lname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))
        

### Validate our Data Model using a SELECT

In [10]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
val_query = "SELECT artist_name, song_title, fname, lname FROM artist_song_user_table WHERE user_id = 10 AND session_number = 182"
try:
    results = session.execute(val_query)
except Exception as e:
    print(e)
    
for result in results:
    print ("Artist: {}\nSong: {}\nName: {} {} ({}, {})\n".format(result.artist_name,
                                                      result.song_title,
                                                      result.fname,
                                                      result.lname, 
                                                      result.fname,
                                                      result.lname))
    

Artist: Down To The Bone
Song: Keep On Keepin' On
Name: Sylvie Cruz (Sylvie, Cruz)

Artist: Three Drives
Song: Greece 2000
Name: Sylvie Cruz (Sylvie, Cruz)

Artist: Sebastien Tellier
Song: Kilometer
Name: Sylvie Cruz (Sylvie, Cruz)

Artist: Lonnie Gordon
Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
Name: Sylvie Cruz (Sylvie, Cruz)



## Query 3

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'

#### Table 3
**Columns:** 
1. fname
2. lname
3. song_title
4. user_id

**Primary Key:**
- (song_title, user_id) : user_id is used along with song_title as song_title is not unique and user_id is used to associate users with the song title.

In [11]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command   

table3 = "user_song_table"
query = "CREATE TABLE IF NOT EXISTS {}".format(table3)
query = query + "(fname text, lname text, song_title text, user_id int, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO user_song_table (fname, lname, song_title, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[1], row[4], row[9], int(row[10])))

### Validate our Data Model using a SELECT

In [13]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
val_query = "SELECT fname, lname FROM user_song_table WHERE song_title = 'All Hands Against His Own'"
try:
    results = session.execute(val_query)
except Exception as e:
    print(e)
    
for result in results:
    print ("Name: {} {} (First Name: {}, Last Name: {})\n".format(result.fname, result.lname,
                                                      result.fname,
                                                      result.lname))

Name: Jacqueline Lynch (First Name: Jacqueline, Last Name: Lynch)

Name: Tegan Levine (First Name: Tegan, Last Name: Levine)

Name: Sara Johnson (First Name: Sara, Last Name: Johnson)



### Drop the tables before closing out the sessions

In [14]:
tables = [table1, table2, table3]

for table in tables:
    query = "drop table {}".format(table)
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()